In [23]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from scipy.sparse import hstack
from collections import Counter
from autosklearn import classification, metrics
import pickle

pd.set_option('max_colwidth', -1)

In [30]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat']

In [4]:
data_path = Path('../data')
train = pd.read_csv(Path(data_path, 'normalized_train.csv')).fillna(' ')

In [7]:
test = train.sample(1000, random_state=31)

In [13]:
test_text = test['comment_text']

In [11]:
with open('../data/word_vectorizer.pkl', 'rb') as file:
    word_vectorizer = pickle.load(file)

In [12]:
with open('../data/char_vectorizer.pkl', 'rb') as file:
    char_vectorizer = pickle.load(file)

In [14]:
test_word_features = word_vectorizer.transform(test_text)
test_char_features = char_vectorizer.transform(test_text)

In [15]:
test_features = hstack([test_char_features, test_word_features])

In [10]:
with open('../data/automl_classifiers_toxic.pkl', 'rb') as file:
    toxic_cf = pickle.load(file)

In [16]:
y_preds = toxic_cf.predict_proba(test_features)[:, 1]

In [21]:
y_labels_preds = toxic_cf.predict(test_features)

In [31]:
for class_name in class_names:
    print("Evaluating {}".format(class_name))
    print()
    with open('../data/automl_classifiers_{}.pkl'.format(class_name), 'rb') as file:
        cf = pickle.load(file)
    y_preds = cf.predict_proba(test_features)[:, 1]
    y_labels_preds = cf.predict(test_features)
    print("ROC AUC: ", roc_auc_score(test[class_name], y_preds))
    print("ACC SCORE: ", accuracy_score(test[class_name], y_labels_preds))
    print(classification_report(test[class_name], y_labels_preds, target_names=['NON_{}'.format(class_name), class_name], digits=3))

[WARNING] [15:22:13:autosklearn.util.backend] Could not delete output dir: /tmp/autosklearn_output_15704_6339
[WARNING] [15:22:13:autosklearn.util.backend] Could not delete tmp dir: /tmp/autosklearn_tmp_15704_6339


Evaluating toxic



[WARNING] [15:22:14:autosklearn.util.backend] Could not delete output dir: /tmp/autosklearn_output_15704_8475
[WARNING] [15:22:14:autosklearn.util.backend] Could not delete tmp dir: /tmp/autosklearn_tmp_15704_8475


ROC AUC:  0.9485968954186653
ACC SCORE:  0.953
             precision    recall  f1-score   support

  NON_toxic      0.956     0.995     0.975       913
      toxic      0.900     0.517     0.657        87

avg / total      0.951     0.953     0.947      1000

Evaluating severe_toxic



[WARNING] [15:22:16:autosklearn.util.backend] Could not delete output dir: /tmp/autosklearn_output_15704_9223
[WARNING] [15:22:16:autosklearn.util.backend] Could not delete tmp dir: /tmp/autosklearn_tmp_15704_9223


ROC AUC:  1.0
ACC SCORE:  0.999
                  precision    recall  f1-score   support

NON_severe_toxic      0.999     1.000     0.999       986
    severe_toxic      1.000     0.929     0.963        14

     avg / total      0.999     0.999     0.999      1000

Evaluating obscene



[WARNING] [15:22:18:autosklearn.util.backend] Could not delete output dir: /tmp/autosklearn_output_15704_1420
[WARNING] [15:22:18:autosklearn.util.backend] Could not delete tmp dir: /tmp/autosklearn_tmp_15704_1420


ROC AUC:  0.976749714720426
ACC SCORE:  0.982
             precision    recall  f1-score   support

NON_obscene      0.983     0.999     0.991       956
    obscene      0.964     0.614     0.750        44

avg / total      0.982     0.982     0.980      1000

Evaluating threat

ROC AUC:  1.0
ACC SCORE:  1.0
             precision    recall  f1-score   support

 NON_threat      1.000     1.000     1.000       997
     threat      1.000     1.000     1.000         3

avg / total      1.000     1.000     1.000      1000

